# <center>Распознавание чисел</center>

Задание поделено на 4 логических части:
<br>1.загрузка данных
<br>2.Написание нейронной сети
<br>3.Задание стуктуры нейнорой сети
<br>4.Визуализация

<br>Импорт библиотек для загрузки данных и написания нейронной сети: 

In [34]:
import pickle
import numpy as np
import random

## Загрузка данных

В данной части происходит загрузка и распределение данных из файла. В качестве данных используются образцы рукописного   написания цифр из базы данных MNIST.

In [35]:
def load_data():
    with open('mnist.pkl', 'rb') as f:
        u = pickle._Unpickler(f)
        u.encoding = 'latin1'
        training_data, validation_data, test_data = u.load()
        return (training_data, validation_data, test_data)

Следующим шагом приводим уже распределенные данные к матричному виду.

In [36]:
def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

#приведение результата к вектору
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

В конечном результате присваиваем результат выполнения вышеупомянутой функции следующим переменным.  

In [37]:
training_data, validation_data, test_data = load_data_wrapper()

## Написание нейронной сети

В качестве нс представлен класс Netvork, при обращении в которм в дальнейшем можно задать структуру нс, а также настроить работу градиентного спуска. В составляющие класса входят только базовые, так как упор был на то, чтобы понять работу нейронных сетей на практике и написать работающий алгоритм.

In [38]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
    
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    # градиентный спуск
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data):
        
        #для визуализации
        self.AccList = []   #список изменения точности на протяжении всех эпох
        self.LossList = []  #список изменения ошибки на протяжении всех эпох
        self.EpochsList = []
        
        n_test = None
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        training_data = list(training_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k + mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(
                    "Epoch {0}:\n Accuracy:{1} Loss:{2}".format(
                        j,
                        self.evaluate(test_data),
                        n_test - self.evaluate(test_data)))
                self.AccList.append(self.evaluate(test_data))
                self.LossList.append(n_test - self.evaluate(test_data))
                self.EpochsList.append(j + 1)
            else:
                print(
                    "Epoch {0} complete".format(j))

        self.AccTrain = self.evaluate(test_data) / 100 % 100               #конечный результат точности
        self.LossTrain = (n_test - self.evaluate(test_data)) / 100 % 100   #конечный результат ошибка

        print("Fin:\n Accuracy:{0}% Loss: {1}%".format(
            self.AccTrain,
            self.LossTrain))

    #размеры пакетов
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta / len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]

    
    # обратное распространение ошибки
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        delta = self.cost_derivative(activations[-1], y) * \
                sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

## Задание стуктуры нейнорой сети

Структура данной нейронной сети состоит из 3 слоев:
    <br>    1.первый слой - входной: 784 входа
    <br>    2.второй слой - скрытый: 30 входов
    <br>    3.третий слой - выходной: 10 входов

В следующей строке можно настроисть работу градиентного спуска посредством измененения таких параметров:
    <br>    1.количества эпох
    <br>    2.размера пакетов
    <br>    3.скороти

Кроме этого в качестве аргументов данная функция так же принимате данные для тренировочной и тестовой выборки.

In [39]:
net = Network([784,30,10])
net.SGD(training_data, 30, 10, 4.0, test_data)
AccList = net.AccList
LossList = net.LossList
epochs = net.EpochsList

Epoch 0:
 Accuracy:8281 Loss:1719
Epoch 1:
 Accuracy:9252 Loss:748
Epoch 2:
 Accuracy:9304 Loss:696
Epoch 3:
 Accuracy:9332 Loss:668
Epoch 4:
 Accuracy:9390 Loss:610
Epoch 5:
 Accuracy:9371 Loss:629
Epoch 6:
 Accuracy:9422 Loss:578
Epoch 7:
 Accuracy:9391 Loss:609
Epoch 8:
 Accuracy:9427 Loss:573
Epoch 9:
 Accuracy:9423 Loss:577
Epoch 10:
 Accuracy:9452 Loss:548
Epoch 11:
 Accuracy:9475 Loss:525
Epoch 12:
 Accuracy:9449 Loss:551
Epoch 13:
 Accuracy:9456 Loss:544
Epoch 14:
 Accuracy:9487 Loss:513
Epoch 15:
 Accuracy:9484 Loss:516
Epoch 16:
 Accuracy:9504 Loss:496
Epoch 17:
 Accuracy:9486 Loss:514
Epoch 18:
 Accuracy:9471 Loss:529
Epoch 19:
 Accuracy:9480 Loss:520
Epoch 20:
 Accuracy:9492 Loss:508
Epoch 21:
 Accuracy:9490 Loss:510
Epoch 22:
 Accuracy:9469 Loss:531
Epoch 23:
 Accuracy:9482 Loss:518
Epoch 24:
 Accuracy:9485 Loss:515
Epoch 25:
 Accuracy:9498 Loss:502
Epoch 26:
 Accuracy:9507 Loss:493
Epoch 27:
 Accuracy:9473 Loss:527
Epoch 28:
 Accuracy:9471 Loss:529
Epoch 29:
 Accuracy:950

Результат: 95.06% точности и 4.94% ошибки

## Визуализация

Для визуализации была использованна графическая библиотека Plotly. 

In [40]:
import plotly
from plotly.offline import * 
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
import pandas as pd

init_notebook_mode(connected = True)

График роста точности: 

In [41]:
iplot([{"x":epochs,"y":AccList}]) 

График снижения ошибки:

In [42]:
iplot([{"x":epochs,"y":LossList}])

График относительно роста точности и снижения ошибки:

In [43]:
trace = go.Scatter3d(
            x = epochs,
            y = AccList,
            z = LossList,
            mode = 'markers',
            marker = dict(
                size = 7,
                color=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
                colorscale = 'Viridis',
                opacity = 0))
layout = go.Layout(
            margin = dict(
                l = 0,
                r = 0,
                b = 0,
                t = 0))
fig = go.Figure(data = [trace], layout = layout)
iplot(fig,filename = '3d-visualisation')

### <center>Спасибо за внимание!</center> 